In [1]:
import numpy as np
import pandas as pd
import re

### Understanding the data.

In [2]:
data=pd.read_csv('Bengaluru_House_Data.csv')

In [3]:
data.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [4]:
data.shape

(13320, 9)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [6]:
for column in data.columns:
    print(data[column].value_counts())
    print('*'*20)

area_type
Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: count, dtype: int64
********************
availability
Ready To Move    10581
18-Dec             307
18-May             295
18-Apr             271
18-Aug             200
                 ...  
16-Oct               1
17-Jan               1
16-Nov               1
16-Jan               1
14-Jul               1
Name: count, Length: 81, dtype: int64
********************
location
Whitefield                         540
Sarjapur  Road                     399
Electronic City                    302
Kanakpura Road                     273
Thanisandra                        234
                                  ... 
3rd Stage Raja Rajeshwari Nagar      1
Chuchangatta Colony                  1
Electronic City Phase 1,             1
Chikbasavanapura                     1
Abshot Layout                        1
Name: count, Length: 1305, dtype: int64
********************
siz

In [7]:
data.isna().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

### Cleaning the data.

In [8]:
data.drop(columns=['area_type', 'society', 'balcony', 'availability'], inplace=True)

In [9]:
data.describe()

,bath,price
count,13247.000000,13320.000000
mean,2.692610,112.565627
std,1.341458,148.971674
min,1.000000,8.000000
25%,2.000000,50.000000
50%,2.000000,72.000000
75%,3.000000,120.000000
max,40.000000,3600.000000


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13319 non-null  object 
 1   size        13304 non-null  object 
 2   total_sqft  13320 non-null  object 
 3   bath        13247 non-null  float64
 4   price       13320 non-null  float64
dtypes: float64(2), object(3)
memory usage: 520.4+ KB


In [11]:
data.dropna(subset=['location'],inplace=True)

In [12]:
def extract_bhk(size_str):
    try:
        return int(size_str.split(' ')[0])
    except:
        return np.nan
    
data['bhk'] = data['size'].apply(extract_bhk)

In [13]:
def impute_bhk(row):
    price = row['price']
    low, high = price * 0.9, price * 1.1
    similar = data[(data['price'] >= low) & (data['price'] <= high) & (data['bhk'].notnull())]
    
    if similar.shape[0] == 0:
        return data['bhk'].median()
    
    return round(similar['bhk'].median())

data.loc[data['bhk'].isnull(), 'bhk'] = data[data['bhk'].isnull()].apply(impute_bhk, axis=1)

In [14]:
data=data[data['bhk']<=7]

In [15]:
def impute_bath(row):
    price = row['price']
    low, high = price * 0.9, price * 1.1
    similar = data[(data['price'] >= low) & (data['price'] <= high) & (data['bath'].notnull())]
    
    if similar.shape[0] == 0:
        return data['bath'].median()
    
    return round(similar['bath'].median())

data.loc[data['bath'].isnull(), 'bath'] = data[data['bath'].isnull()].apply(impute_bath, axis=1)

In [16]:
def convertRange(x):
    temp=x.split('-')
    if len(temp)==2:
        return (float(temp[0])+float(temp[1]))/2
    try:
        return float(x)
    except:
        return float(re.match(r'(^\d+)',x)[0])

In [17]:
data['total_sqft'] = data['total_sqft'].apply(convertRange)

In [18]:
data['price_per_sqft']=data['price']*100000/data['total_sqft']

In [19]:
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13150.000000,13150.000000,13150.000000,13150.000000,1.315000e+04
mean,1543.916862,2.617643,110.773471,2.721673,1.287326e+04
std,1172.762676,1.095608,144.392454,1.016456,2.599789e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.262295e+03
50%,1275.000000,2.000000,70.190000,3.000000,5.416667e+03
75%,1670.000000,3.000000,119.750000,3.000000,7.261467e+03
max,52272.000000,9.000000,2912.000000,7.000000,2.000000e+07


In [20]:
data['location']=data['location'].apply(lambda x: x.strip())
location_count = data['location'].value_counts()

In [21]:
location_count_less_10=location_count[location_count<=10]

In [22]:
data['location'] = data['location'].apply(lambda x: 'other' if x in location_count_less_10 else x)

In [23]:
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13150.000000,13150.000000,13150.000000,13150.000000,1.315000e+04
mean,1543.916862,2.617643,110.773471,2.721673,1.287326e+04
std,1172.762676,1.095608,144.392454,1.016456,2.599789e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.262295e+03
50%,1275.000000,2.000000,70.190000,3.000000,5.416667e+03
75%,1670.000000,3.000000,119.750000,3.000000,7.261467e+03
max,52272.000000,9.000000,2912.000000,7.000000,2.000000e+07


In [24]:
(data['total_sqft']/data['bhk']).describe()

count    13150.000000
mean       577.333931
std        386.632804
min          0.250000
25%        475.000000
50%        554.000000
75%        625.500000
max      26136.000000
dtype: float64

In [25]:
data=data[((data['total_sqft']/data['bhk'])>=300)]
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,12486.000000,12486.000000,12486.000000,12486.000000,12486.000000
mean,1580.070938,2.539885,110.280753,2.626141,6296.172644
std,1189.684165,1.000666,147.219514,0.890156,4135.040307
min,300.000000,1.000000,8.440000,1.000000,267.829813
25%,1115.000000,2.000000,49.000000,2.000000,4210.526316
50%,1300.000000,2.000000,70.000000,3.000000,5294.736984
75%,1700.000000,3.000000,115.000000,3.000000,6911.465806
max,52272.000000,9.000000,2912.000000,7.000000,176470.588235


In [26]:
data['price_per_sqft'].describe()

count     12486.000000
mean       6296.172644
std        4135.040307
min         267.829813
25%        4210.526316
50%        5294.736984
75%        6911.465806
max      176470.588235
Name: price_per_sqft, dtype: float64

In [27]:
def remove_outliers_sqft(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf['price_per_sqft'])
        st = np.std(subdf['price_per_sqft'])
        reduced_df = subdf[(subdf['price_per_sqft']>(m-st)) & (subdf['price_per_sqft']<=(m+st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index=True)
    return df_out

data=remove_outliers_sqft(data)
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10274.000000,10274.000000,10274.000000,10274.000000,10274.000000
mean,1500.009315,2.458536,90.742475,2.556161,5650.797416
std,860.945090,0.915427,85.553143,0.820743,2235.894980
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1108.000000,2.000000,49.000000,2.000000,4244.994224
50%,1283.500000,2.000000,67.000000,2.000000,5176.116129
75%,1650.000000,3.000000,100.000000,3.000000,6426.000592
max,30400.000000,9.000000,2200.000000,7.000000,24000.000000


In [28]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df['price_per_sqft']),
                'std': np.std(bhk_df['price_per_sqft']),
                'count': bhk_df.shape[0]
            }
        
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df['price_per_sqft'] < (stats['mean'] - stats['std'])].index.values)
    return df.drop(exclude_indices, axis='index')

In [29]:
data=bhk_outlier_remover(data)

In [30]:
data.shape

(9505, 7)

In [31]:
data.drop(columns=['size', 'price_per_sqft'], inplace=True)

In [32]:
data.to_csv('cleaned_data.csv', index=False)